## Tuesday (01/19) Meeting
#### These are covered in the network_generation notebook, intro here?:
- How to generate a control network purely in autocnet 
    - Sub-pixel registration 
        - Custom cost functions
    - Cross-instrument matching   
        - Custom cost functions

I would like to have a database that we can look at apriori and adjusted ROIs (before and after subpixel registration). Should we have another demo database, totally static, or just have them edit the database name they use when generating their networks (workshop_kaguya_reiner_gamma_NAME) so we can use the one I am generating as a part of the network_generation notebook?

    
#### Covered in notebooks from last sprint:
- How to ingest an ISIS control network 
- Overlap analysis
- Outlier detection 

# AutoCNet Intro
AutoCNet is a suite of functions that parallelize network generation and analyze the health of networks. Networks within AutoCNet are represented with an [undirected graph](https://en.wikipedia.org/wiki/Graph_(discrete_mathematics)), called a NetworkCandidateGraph. AutoCNet leverages a database for presistent storage of the images, points, and measures of a network. The use of this database allows for quick access of each elements geometries and how those geometries relate with one another. 

**[insert flow chart of AutoCNet? Making sure to highlight the placement of the services that require config variables]**

# AutoCNet Configuration
AutoCnet leverages numerous services for the cluster processing and the data presistance. This services require configuration parameters at the creation of the NetworkCandidateGraph. These parameters are typically held in a configuration yaml file. 

The services leverage in a network are:
- cluster of choice (ASC internal Nebula)

There corresponding necessary configuration values are:
- cluster: meta configurations for the cluster
    - maxfailures [int]: the number of times to retry a failing cluster job (default)
    - queue [str]: name of the processing queue to be used for cluster jobs
    - cluster_log_dir [str]: path location to put log files of the cluster jobs
    - cluster_submission [str]: The name of the job scheduler used on the cluster (e.g.: 'slurm' or 'pbs')
    - extractor_memory [int]: The amount of RAM to request for jobs (in MB)
    - processing_memory [int]: The amount of RAM to request for jobs (in MB)
- env: the conda and ISIS environment configuration to be used on the cluster jobs
    - conda [str]: name of the conda environment to activate before cluster jobs execution
    - ISISROOT [str]: path to the ISISROOT directory
    - ISISDATA [str]: path to the ISISDATA directory
- database: meta configuration for the database
    - type [str]: name of the database management system (e.g.: 'postgresql' or 'sqlite')
    - username [str]: username used to login to the database
    - password [str]: password used to login to the database
    - host [int]: name of the host (or local) machine running the database
    - port [int]: port that the database is exposed on
    - pgbouncer_port [int]: port that is running the proxy that manages database connections
    - name [str]: name of database where tables will be created. PostGreSQL requires all lowercase database names.
    - timeout [int]: number of seconds to wait while attempting to connect to database

- redis
- spatial
- directories

Do not require change
- pfeffernuss



redis:
  51     # Name of the redis queue. Ensure it is unique to avoid multiple uses
  52     # of the same redis queue.
  53     basename: 'kag_rg'
  54     # Hostname (or localhost) where the redis server is running
  55     host: 'autocnet.wr.usgs.gov'
  56     # Port that redis is accepting connections on
  57     port: '30009'
  58     # The name of the queue that successful job messages are pushed. This
  59     # name should be unique to avoid collision with other users.
  60     completed_queue: 'kag_rg:done'
  61     # The name of the queue used for jobs that need to be started. This
  62     # name should be unique to avoid collision with other users.
  63     processing_queue: 'kag_rg:proc'
  64     # The name of the queue that currently processing jobs are pushed to. This
  65     # name should be unique to avoid collision with other users.
  66     working_queue: 'lak_kag_rg:working'


spatial:
    
    target: 'MOON' # The name of the target body that will be output in ISIS control networks.
    # The following two values need to be added to the database that is storing
    # the spatial information.
    #
    # The spatial reference identifier for latitudinal geometries in the database.
    latitudinal_srid: 930100
    # The spatial reference identifier for rectangular geometries in the database.
    rectangular_srid: 930180
    # The radii of the body. If the body is a sphere, these values should match.
    # Units are in meters
    semimajor_rad: 1737400
    semiminor_rad: 1737400
    # A compliant proj4 string. Checkout http://spatialreference.org for examples
    # for planetary bodies.
    proj4_str: '+proj=longlat +a=1737400 +b=1737400 +no_defs'
    # DEM used to generate height values for projection
    dem: '/usgs/cpkgs/isis3/isis_data/base/dems/ldem_128ppd_Mar2011_clon180_radius_pad.cub

In [ ]:
config_path = f'/home/ladoramkershner/projects/config_autocnet/moon/kaguya/workshop/kaguya_reiner_gamma_morning_workshop.yml'

In [ ]:
from autocnet.config_parser import parse_config

config = parse_config(config_path)

Check out different configuration variables

In [ ]:
print(config['spatial'])

# Components of The NetworkCandidateGraph
The network candidate graph is an implimentation of an undirected graph, which contains nodes and edges. A node in a graph is synonymous with an image. The node (image) stores PATH information, an accessor to the on-disk data set, and correspondences information that references the image. An edge in a graph represent the relationships between the nodes (images); it contains information such as the source and destination image ids (and associated node ids), the overlap dimensions, and points/measures shared between the two nodes (images).



The 'Network' in NetworkCandidateGraph references a database component which contains the presisted information of an AutoCNet network.

# Database Table Structures
The database and the AutoCNet network information that is presisted are organized into the following tables:
- cameras
- costs
- edges
- images
- keypoints
- matches
- measures
- overlay 
- points

### Cameras
The camera tables holds the informatiuon relevant to the amera models used in a control network.
- id: primary key index for the cameras table 
- image_id: foreign key pointing to the primary key index of the 'images' table
- camera: ?????????
- cam_type: type of camera model used; current options include ['csm', 'isis']

**Are these kinds of database table descriptions helpful?**

# Network Generation Intro

What is the process?
- Set up configuration so you have access to all components of cluster enable AutoCNet
- Lay down points
- Subpixel register relatively
    - Put a visualization of the ROIs before and after subpixel registration
- Iterate to a good network
    - Jigsaw still done in ISIS (CSM bundle under construction)
- Groujnd strong relative network using cross-instrument matcher

# Cluster Processing 
 Cluster pricessing in AutoCNet is called using NetworkCandidateGraph.apply() function 
 
 (function,
                            on='edges',
                            filters,
                            query_string,
                            args,
                            walltime,
                            chunksize,
                            arraychunk,
                            reapply,
                            log_dir)

In [ ]:
from autocnet.graph.network import NetworkCandidateGraph
ncg = NetworkCandidateGraph()
ncg.apply?